In [15]:
import kfp

from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

In [20]:
authservice_session='authservice_session=MTYxMzc1NDU4MXxOd3dBTkZKWFJEWlRVMEpHUnpKT1RFMVlOa2MyVURWRVQwUkxNa2REUWpjelJGWlNNMGhLV0VsSk5VZFdSMEZXTWpVMVZFTkxVMEU9fI-9lqJWwao6cf2fWLz0HHdd2Pqqk_FtMgeJMPjDaPeT'
client = kfp.Client(host='http://a8a635fce30784afb9986f2e5aec7888-657378971.us-west-2.elb.amazonaws.com/pipeline', cookies=authservice_session)
client.list_experiments(namespace="admin")

{'experiments': [{'created_at': datetime.datetime(2021, 2, 10, 13, 41, 52, tzinfo=tzlocal()),
                  'description': None,
                  'id': 'bdeff48a-9724-4946-91f0-75f7f6f27552',
                  'name': 'test',
                  'resource_references': [{'key': {'id': 'admin',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'},
                 {'created_at': datetime.datetime(2021, 2, 11, 6, 2, 7, tzinfo=tzlocal()),
                  'description': None,
                  'id': '08aa6c17-897f-4e38-8064-9b0ca4a6708f',
                  'name': 'sample',
                  'resource_references': [{'key': {'id': 'admin',
                                                   'type': 'NAMESPACE'},
                                           'name': None,
               

In [21]:
model="resnet"

In [22]:
# load components (note the components are not platform specific, but the importers are)
data_prep_op = load_component_from_file(f"data_prep_step/{model}/component.yaml")
train_model_op = load_component_from_file(f"training_step/{model}/component.yaml")
list_item_op = load_component_from_file("file/component.yaml")
download_op = load_component_from_file("web/component.yaml")
model_archive_op = load_component_from_file("model_archive_step/component.yaml")

In [23]:
@dsl.pipeline(
    name = "pytorchcnn"
)
def train_imagenet_cnn_pytorch(
    ):
        
    # data_prep_task = data_prep_op(input_data = "")
    # data_prep_task = data_prep_op(input_data = "gs://cloud-ml-nas-public/classification/imagenet/train*", vocab_file = "bert_base_uncased_vocab.txt", vocab_file_url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt")

    # temp_input = "gs://managed-pipeline-test-bugbash/20210130/pipeline_root/pavel/c14ec128-18d4-4980-b9f3-e1c6f4babb51/pytorchcnn-dj5sg-2878573190/output_data/prefix"
    #data_prep_task.outputs["output_data"])

    # train_model_task = (train_model_op(trainingdata = "").
    #     set_cpu_limit('4').
    #     set_memory_limit('14Gi')
    # )    
    # archive_prep_task = archive_prep_op(folder = train_model_task.outputs["modelcheckpoint"], model_name = "resnet")

    # train_model_task = (train_model_op(trainingdata = data_prep_task.outputs["output_data"], maxepochs = 2, 
    #     numsamples = 150, 
    #     batchsize = 16,
    #     numworkers = 2,
    #     learningrate = 0.001,
    #     accelerator = "")
    #     .set_cpu_limit('4').
    #     set_memory_limit('14Gi')
    # )

    download_task0 = download_op(url = "https://download.pytorch.org/models/resnet18-5c106cde.pth", input = "")
    download_task1 = download_op(url = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/properties.json", input = "")
    download_task2 = download_op(url = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/index_to_name.json", input = download_task1.outputs["output"])
#     download_task3 = download_op(url = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model.py", input = download_task2.outputs["output"])
    list_item_task = list_item_op(directory = download_task0.outputs["output"])

    model_archive_task = model_archive_op(properties=download_task1.outputs["output"], serializedfile = download_task0.outputs["output"], extrafiles = download_task2.outputs["output"])

    list_item_task = list_item_op(directory = model_archive_task.outputs["output_directory"])

#     deploy_model_task = deploy_model_op(
#         action = 'create',
#         model_name='pytorch',
#         default_model_uri='gs://kfserving-samples/models/pytorch/cifar10/',
#         namespace='admin',
#         framework='pytorch',
#         default_custom_model_spec='{}',
#         canary_custom_model_spec='{}',
#         autoscaling_target='0',
#         kfserving_endpoint=''
#     )


In [24]:
compiler.Compiler().compile(train_imagenet_cnn_pytorch, 'pytorch.tar.gz')
# Execute pipeline
run = client.run_pipeline("9b0390ee-a3cf-4070-afdf-1c1b7392cb8b", 'pytorch-model', 'pytorch.tar.gz')